In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

In [2]:
train = pd.read_csv("../input/titanic/train.csv")
train.sample(6)

In [3]:
x_train = train.drop(columns=["Survived"])
y_train = train["Survived"]

In [4]:
numerical_features = ["Age", "SibSp", "Parch", "Fare"]
categorical_features = ["Pclass", "Sex", "Embarked"]

In [5]:
titles = {
    part for name in x_train["Name"] for part in name.split() if part.endswith(".")
}

def preprocess(x):
    for title in titles:
        x[title] = x["Name"].map(lambda x: int(title in x))
        
    x = x.drop(columns=["Name", "Cabin", "Ticket"])
    
    x["Age"] = x["Age"].fillna(x["Age"].median())
    
    x = pd.get_dummies(x, columns=categorical_features)
    return x.drop(columns="PassengerId"), x["PassengerId"]

In [6]:
x_train, train_id = preprocess(x_train)

In [7]:
normalize = tf.keras.layers.Normalization()
normalize.adapt(x_train[numerical_features])

In [8]:
def as_normalized_tensor(x):
    x_numerical = normalize(x[numerical_features])
    x_categorical = tf.convert_to_tensor(
        x.drop(columns=numerical_features), dtype=tf.float32
    )
    return tf.concat([x_numerical, x_categorical], axis=1)

In [9]:
x_train = as_normalized_tensor(x_train)

In [10]:
#x_train = tf.data.Dataset.from_tensor_slices(x_train)

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1)
])
criterion = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(
    optimizer="adam",
    loss=criterion,
    metrics=[tf.keras.metrics.BinaryAccuracy()]
)

In [12]:
model.fit(x=x_train, y=y_train, epochs=15)

In [13]:
x_test = pd.read_csv("../input/titanic/test.csv")
x_test, test_id = preprocess(x_test)
x_test = as_normalized_tensor(x_test)

In [14]:
y_pred = tf.math.sigmoid(model(x_test))
y_pred = tf.cast(tf.math.round(tf.squeeze(y_pred)), dtype=tf.int64)

submission = pd.DataFrame({
    "PassengerId": test_id,
    "Survived": y_pred
})
submission.to_csv("submission.csv", index=False)